In [1]:
# Only needed in pycharm
import os
os.chdir('/home/cs-folq1/src/sts_bert/sBERT')

In [2]:
import torch
from torch.utils.data import DataLoader

from BiEncoder import BiEncoder
from CrossEncoder import CrossEncoder
from STSTrainer import STSTrainer
from Datasets import load_sts
from GridRun import GridRun, random_sample

In [3]:
dataset = load_sts()

Reusing dataset stsb_multi_mt (/home/cs-folq1/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/bc6de0eaa8d97c28a4c22a07e851b05879ae62c60b0b69dd6b331339e8020f07)


In [4]:
def run_experiment(config):
    subset_indices = random_sample(n=len(dataset['train']), k=config['train_size'],
                                   seed=config['train_subset_seed'])
    train_dataset_subset = torch.utils.data.Subset(dataset['train'], subset_indices)

    encoder, mode = config['mode'].split('/')
    if encoder == 'bi-encoder':
        model = BiEncoder(mode=mode, head='cos-sim')
    else:
        model = CrossEncoder(mode=mode)

    trainer = STSTrainer(model=model, train_dataset=train_dataset_subset, dataset=dataset,
                         num_epochs=config['num_epochs'], batch_size=config['batch_size'],
                         lr=config['lr'], lr_scheduler=config['lr_scheduler'],
                         warmup_percent=config['warmup_percent'])
    result = trainer.train(disable_progress_bar=False)
    save_name = encoder + '_' + mode
    return result, model, save_name


grid = {
    'num_epochs': 10,
    'batch_size': 16,
    'lr': 2e-5,
    'lr_scheduler': 'linear',
    'warmup_percent': 0.2,
    'mode': [
             'bi-encoder/nli-cls-pooling',
             'bi-encoder/nli-mean-pooling',
             'bi-encoder/nli-linear-pooling'],
    #'train_size': [500, 1000, 2000, 3000, 4000, len(dataset['train'])],
    'train_size': [len(dataset['train'])],
    'train_subset_seed': [1, 2, 3]
}

grid_run = GridRun(run_experiment, results_dir='results', experiment_name='pretraining_sts')
#grid_run = GridRun(run_experiment)
grid_run.run(grid, save_best=True, ignore_previous_results=True)
# grid_run.df_results

Loading previous results from results/pretraining_sts/results_210707_152607.csv
Results will be stored in file results_210708_100708
Already done: {'batch_size': [16.0], 'lr': [2e-05], 'lr_scheduler': ['linear'], 'mode': ['bi-encoder/nli-cls-pooling'], 'num_epochs': [10.0], 'test_score': [0.8376018218342469], 'train_size': [5749.0], 'train_subset_seed': [1.0], 'warmup_percent': [0.2]}
Repeating experiment
----------
RUN CONFIG
----------
num_epochs :  10
batch_size :  16
lr :  2e-05
lr_scheduler :  linear
warmup_percent :  0.2
mode :  bi-encoder/nli-cls-pooling
train_size :  5749
train_subset_seed :  1


  0%|          | 0/360 [00:00<?, ?it/s]

Scheduler type: linear, epochs: 10, steps per epoch: 360, total steps: 3600, warmup steps: 720


  0%|          | 0/360 [00:00<?, ?it/s]

Epoch 1   : loss: 0.0378  , score: 0.8405  *


...

Epoch 4   : loss: 0.0532  , score: 0.8091  *


  0%|          | 1/360 [00:00<00:46,  7.75it/s]

Epoch 5   : loss: 0.0515  , score: 0.8142  *


 57%|█████▋    | 205/360 [00:28<00:21,  7.29it/s]


KeyboardInterrupt: 

In [4]:
sts_model = CrossEncoder(mode='cls-pooling-hidden', toy_model=False)
sts_trainer = STSTrainer(model=sts_model, train_dataset=dataset['train'],
                         dataset=dataset, num_epochs=10, lr_scheduler='linear',
                         warmup_percent=0.2)
#sts_trainer.train(disable_progress_bar=True)
original_sts_model_path = './saved_models/few_shot_paws/original_cross_encoder.bin'
# torch.save(sts_model.state_dict(), original_sts_model_path)
sts_model.load_state_dict(torch.load(original_sts_model_path))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scheduler type: linear, epochs: 10, steps per epoch: 360, total steps: 3600, warmup steps: 720


<All keys matched successfully>

In [6]:
sts_trainer.score(sts_trainer.test_dl)

100%|██████████| 87/87 [00:03<00:00, 27.72it/s]


(0.852038992078638, tensor(0.0240))